In [ ]:
from google.colab import files
import pickle as pkl
import pandas as pd
def preprocess_df(df):
    df['state']=[st.lower() for st in df['state']]
    df['categoryName']=[cat.lower() for cat in df['categoryName']]
    df['city']=[ct.lower() for ct in df['city']]
    return df

uploads=files.upload()   
df = pd.read_excel('refined_dataset.xlsx') 
df = preprocess_df(df)

Saving refined_dataset.xlsx to refined_dataset (1).xlsx


In [ ]:
class category_index:# class for the inverted index storing tourist attractions' indices based on category
  def __init__(self,df):
    self.dataframe=df
    self.index_dict=dict()
    self.cat_list=[st.lower() for st in df['categoryName'].unique()]

  def get_cat_index(self,cat):
    cat_df=self.dataframe[self.dataframe['categoryName']==cat]# getting the indices based on a given category
    return list(cat_df.index.values)

  def make_index(self):
    for cat in self.cat_list:
      self.index_dict[cat.lower()]=self.get_cat_index(cat)# making index
      
  def get_cat_list(self):
    return self.cat_list


In [ ]:
class state_index:# class for the inverted index storing tourist attractions' indices based on state
  def __init__(self,df):
    self.dataframe=df
    self.index_dict=dict()
    self.state_list=[st.lower() for st in df['state'].unique()]

  def get_st_index(self,st):# getting the indices based on a given state
    st_df=self.dataframe[self.dataframe['state']==st]
    return list(st_df.index.values)

  def make_index(self):# making index
    for st in self.state_list:
      self.index_dict[st.lower()]=self.get_st_index(st)

  def get_state_list(self):
    return self.state_list


In [ ]:
class city_index:# class for the inverted index storing tourist attractions' indices based on city
  def __init__(self,df):
    self.dataframe=df
    self.index_dict=dict()
    self.city_list=[st.lower() for st in df['city'].unique()]

  def get_city_index(self,ct):# getting the indices based on a given city
    ct_df=self.dataframe[self.dataframe['city']==ct]
    return list(ct_df.index.values)

  def make_index(self):# making index
    for ct in self.city_list:
      self.index_dict[ct.lower()]=self.get_city_index(ct)

  def get_ct_list(self):
    return self.city_list


In [ ]:
from math import sin,cos,asin,radians,sqrt
import numpy as np
class distance_index:
  def __init__(self,df):
    self.df=df
    self.index=dict()
    self.near_by_place=dict()
    self.THRESHOLD=200
  def get_distance(self,lat1,lat2,long1,long2):
    lon1 = radians(long1)
    lon2 = radians(long2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371      
    # calculate the result
    return(c * r)
  def make_index_keys(self):
    places=self.df['title']
    indx=self.df.index
    for i in range(len(places)-1):
      for j in range(1,len(places)):
        lat1=self.df.at[i,'location_latitude']
        lat2=self.df.at[j,'location_latitude']
        long1=self.df.at[i,'location_longitude']
        long2=self.df.at[j,'location_longitude']
        dist=self.get_distance(lat1,lat2,long1,long2)
        pl1=places[i]
        pl2=places[j]
        tup=tuple([pl1,pl2])
        self.index[tup]=dist
  def make_nearby_places(self):
    for pl in self.df['title']:
      self.near_by_place[pl]=list()
      for key in self.index.keys():
        if pl==key[0] and self.index[key]<=self.THRESHOLD:
          self.near_by_place[pl].append(key[1])
        elif pl==key[1] and self.index[key]<=self.THRESHOLD:
          self.near_by_place[pl].append(key[0])

  def make_nearby_places2(self):
    keys = list(self.index.keys())
    values = list(self.index.values())
    sorted_value_index = np.argsort(values)
    sorted_dict = {keys[i]: values[i] for i in sorted_value_index}
    for pl in self.df['title']:
      self.near_by_place[pl]=list()
    for key in sorted_dict:
      if sorted_dict[key]<=200:
        self.near_by_place[key[0]].append(key[1])
        self.near_by_place[key[1]].append(key[0])
      else:
        break
    








In [ ]:
di=distance_index(df)
di.make_index_keys()
di.make_nearby_places2()

In [ ]:
#print(di.index.items()[0:10])
print(dict(list(di.index.items())[0: 10]))
print(dict(list(di.near_by_place.items())[0: 10]))
import pickle as pkl
foj=open("near_by_places.pkl",'wb')
pkl.dump(di,foj)
foj.close()

{('Gadeshwar Lake', 'Mandapeshwar Caves'): 47.54943535564373, ('Gadeshwar Lake', 'Mani Bhavan Gandhi Sangrahalaya'): 46.400096677700134, ('Gadeshwar Lake', 'Chhota Kashmir Boat Club'): 41.78735293601677, ('Gadeshwar Lake', 'Ghodbunder Fort'): 47.68957268948668, ('Gadeshwar Lake', 'Raigad Fort'): 91.14589196640935, ('Gadeshwar Lake', 'Bird Valley Udyan'): 71.68325142346072, ('Gadeshwar Lake', 'Bhayandar Khadi'): 51.704225313287, ('Gadeshwar Lake', 'Umbrella Fall, Bhandardara'): 78.85782272597304, ('Gadeshwar Lake', 'Sydney Point'): 135.35245308166188, ('Gadeshwar Lake', 'Suvarna Temple'): 36.721187249950205}
{'Gadeshwar Lake': ['Mandapeshwar Caves', 'Mani Bhavan Gandhi Sangrahalaya', 'Chhota Kashmir Boat Club', 'Ghodbunder Fort', 'Raigad Fort', 'Bird Valley Udyan', 'Bhayandar Khadi', 'Umbrella Fall, Bhandardara', 'Sydney Point', 'Suvarna Temple', 'Alibaba Aur Chalis Chorr', 'Pagoda Temple', 'Smart Ecopark', 'Pratapgad Fort', 'Aqua Imagicaa', 'From locals @ Rs.499 Prabalmachi Camping /Ka

In [ ]:
import pandas as pd
from google.colab import files
import pickle as pkl

uploads=files.upload()        # upload the excel dataset and objects stored in pickle format                       
df = pd.read_excel('Refined_dataset.xlsx') 
df = preprocess_df(df)
cai = category_index(df)
cai.make_index()
fobj=open('cat_index.pkl','wb')
pkl.dump(cai,fobj)
fobj.close()

si = state_index(df)
si.make_index()
fobj1=open('state_index.pkl','wb')
pkl.dump(si,fobj1)
fobj1.close()

ci = city_index(df)
ci.make_index()
fobj2=open('city_index.pkl','wb')
pkl.dump(ci,fobj2)
fobj2.close()


files.download('cat_index.pkl')     
files.download('state_index.pkl')     
files.download('city_index.pkl')  


"\nfiles.download('cat_index.pkl')     \nfiles.download('state_index.pkl')     \nfiles.download('city_index.pkl')  \n"

In [ ]:
from google.colab import files
uploads=files.upload()

In [ ]:
cat_syn_obj=open('/content/synonym_cat.txt','r')#loading the category synonym text file
cat_syn_text=cat_syn_obj.read()
cat_syn_dict=dict()
lines=cat_syn_text.split(";")
for ln in lines:# creating dictionary to store the synonyms for a category
  ln_spl=ln.split(":")
  print(ln_spl)
  print(ln_spl[1].split(","))
  for wd in ln_spl[1].split(","):
    wd=wd.replace("'","")
    wd=wd.replace("\n","")
    wd=wd.strip()
    cat_syn_dict[str(wd).lower()]=str(ln_spl[0]).replace("\n","").lower()
  cat_syn_dict[str(ln_spl[0]).replace("\n","").lower()]=str(ln_spl[0]).replace("\n","").lower()
st_syn_obj=open('/content/state_acr.txt','r')
st_syn_text=st_syn_obj.read()
st_syn_dict=dict()
lines_st=st_syn_text.split(";")
for ln in lines_st:# creating the dictionary to store the synonyms for the state
  ln_spl=ln.split(":")
  wd1=ln_spl[1].replace('\n','')
  wd1=wd1.strip()
  st_syn_dict[str(wd1).lower()]=str(ln_spl[0]).replace("\n","").lower()
  st_syn_dict[str(ln_spl[0]).replace("\n","").lower()]=str(ln_spl[0]).replace("\n","").lower()

['Pilgrimage', "'Hindu Temple','Buddhist temple','Pagoda','Church','Catholic church','Jain temple','Cathedral','Shrine','Gurudwara','Mosque','Religious,destination','Bahá'í house of worship','Anglican church','Monastery','Meditation center','Place of worship','Pilgrimage place','Catholic cathedral','Basilica','Cemetery','Tomb','Spiritist center','Religious organization','Protestant church','Religious institution','Christian church','Baptist church','Presbyterian church'"]
["'Hindu Temple'", "'Buddhist temple'", "'Pagoda'", "'Church'", "'Catholic church'", "'Jain temple'", "'Cathedral'", "'Shrine'", "'Gurudwara'", "'Mosque'", "'Religious", "destination'", "'Bahá'í house of worship'", "'Anglican church'", "'Monastery'", "'Meditation center'", "'Place of worship'", "'Pilgrimage place'", "'Catholic cathedral'", "'Basilica'", "'Cemetery'", "'Tomb'", "'Spiritist center'", "'Religious organization'", "'Protestant church'", "'Religious institution'", "'Christian church'", "'Baptist church'", "

In [ ]:
print(cat_syn_dict)
print(st_syn_dict)

{'hindu temple': 'pilgrimage', 'buddhist temple': 'pilgrimage', 'pagoda': 'pilgrimage', 'church': 'pilgrimage', 'catholic church': 'pilgrimage', 'jain temple': 'pilgrimage', 'cathedral': 'pilgrimage', 'shrine': 'pilgrimage', 'gurudwara': 'pilgrimage', 'mosque': 'pilgrimage', 'religious': 'pilgrimage', 'destination': 'pilgrimage', 'baháí house of worship': 'pilgrimage', 'anglican church': 'pilgrimage', 'monastery': 'pilgrimage', 'meditation center': 'pilgrimage', 'place of worship': 'pilgrimage', 'pilgrimage place': 'pilgrimage', 'catholic cathedral': 'pilgrimage', 'basilica': 'pilgrimage', 'cemetery': 'pilgrimage', 'tomb': 'pilgrimage', 'spiritist center': 'pilgrimage', 'religious organization': 'pilgrimage', 'protestant church': 'pilgrimage', 'religious institution': 'pilgrimage', 'christian church': 'pilgrimage', 'baptist church': 'pilgrimage', 'presbyterian church': 'pilgrimage', 'pilgrimage': 'pilgrimage', 'beach': 'beach', 'public beach': 'beach', 'island': 'beach', 'beach pavilli

In [ ]:
import pickle as pkl   # synonym pickling
fobj=open("synonym_cat_dict.pkl",'wb')
pkl.dump(cat_syn_dict,fobj)
fobj.close()
fobj1=open("synonym_st_dict.pkl",'wb')
pkl.dump(st_syn_dict,fobj1)
fobj1.close()

In [ ]:
files.download('synonym_cat_dict.pkl')     # downloading the pkl file for storing locally
files.download('synonym_st_dict.pkl')     

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pickle as pkl   
from google.colab import files

uploads=files.upload()                         # upload the two synonym dicts     
f1=open("synonym_cat_dict.pkl",'rb')
cat_dict=pkl.load(f1)                                  # load the pkl file stored earlier
f1.close() 
f2=open("synonym_st_dict.pkl",'rb')
st_dict=pkl.load(f2)                                  # load the pkl file stored earlier
f2.close() 

Saving synonym_st_dict.pkl to synonym_st_dict.pkl
Saving synonym_cat_dict.pkl to synonym_cat_dict.pkl


In [ ]:
class MainIndex:# creating the cross index, it will store all the indices for each of category, city, state. Also it will store
  def __init__(self):# index lists which satisfies intersection of a city/state and category
    self.cat_i=None
    self.st_i=None
    self.city_i=None
    self.cross_index=dict()
    self.cross_list=list()

  def get_indices(self):# loading the index objects
    fobj1=open('cat_index.pkl','rb')
    self.cat_i=pkl.load(fobj1)
    fobj1.close()
    fobj2=open('state_index.pkl','rb')
    self.st_i=pkl.load(fobj2)
    fobj2.close()
    fobj3=open('city_index.pkl','rb')
    self.city_i=pkl.load(fobj3)
    fobj3.close()

  def merger(self,cat,st='',ct=''):# doing the intersection of category and state/city
    if len(ct)>0:
      l1=self.cat_i.get_cat_index(cat)
      l2=self.ct_i.get_ct_index(ct)
      fin_index=set(l1).intersection(set(l2))
      return list(fin_index)
    elif len(st)>0:
      l1=self.cross_index[cat]
      l2=self.cross_index[st]
      fin_index=set(l1).intersection(set(l2))
      return list(fin_index)
    else:
      return []

  def create_cross_index(self):# creating the index , storing category and state combination as c or (category,city) for key in dictionary
    self.get_indices()
    for cat in self.cat_i.get_cat_list():
        self.cross_index[cat]=list()
        self.cross_index[cat].extend(self.cat_i.get_cat_index(cat))
    for st in self.st_i.get_state_list():
      self.cross_index[st]=list()
      self.cross_index[st].extend(self.st_i.get_st_index(st))
    for city in self.city_i.get_ct_list():
        if city in self.st_i.get_state_list():
              self.cross_index[city].extend(self.city_i.get_city_index(city))
        else:
              self.cross_index[city]=list()
              self.cross_index[city].extend(self.city_i.get_city_index(city))
    for cat in self.cat_i.get_cat_list():# making intersection of indices, (category,state)
      for st in self.st_i.get_state_list():
        tup=tuple([cat,st])
        self.cross_index[tup]=self.merger(cat,st)
    for cat in self.cat_i.get_cat_list():# making intersection of indices, (category,city)
      for ct in self.city_i.get_ct_list():
        tup=tuple([cat,ct])
        self.cross_index[tup]=self.merger(cat,ct)

  def index_lookup(self,cat='',pl=''):# facilitating look up of index
    print("category",cat)
    print("place",pl)
    if len(cat)>0:
      if len(pl)>0:
        tup=tuple([cat,pl])
        return self.cross_index[tup]
      else:
        return self.cross_index[cat]
    elif len(pl)>0:
      return self.cross_index[pl]
    else:
      return []


In [ ]:
uploads=files.upload()            # upload cat_index, state_index, city_index
mi = MainIndex()
mi.create_cross_index()
fobj=open('cross_index.pkl','wb')
pkl.dump(mi,fobj)
fobj.close()
files.download('cross_index.pkl')     

In [ ]:
import nltk
import re
import pickle as pkl   
from google.colab import files
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=list(stopwords.words("english"))
uploads=files.upload()        # upload the 4 indices and 2 synonym mapping dicts                    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
class Query_Processing:# class for processing the query
  def __init__(self):
    self.cat_index=None
    self.state_index=None
    self.city_index=None
    self.cross_index = None
    self.res_df = None
    self.st_syn_dict = None
    self.cat_syn_dict = None
    self.bigram = []
  
  def get_indices(self):
    fobj1=open('cat_index.pkl','rb')
    self.cat_index=pkl.load(fobj1)
    fobj1.close()
    fobj2=open('state_index.pkl','rb')
    self.state_index=pkl.load(fobj2)
    fobj2.close()
    fobj3=open('city_index.pkl','rb')
    self.city_index=pkl.load(fobj3)
    fobj3.close()
    fobj4=open('synonym_cat_dict.pkl','rb')
    self.cat_syn_dict=pkl.load(fobj4)
    fobj4.close()
    fobj5=open('synonym_st_dict.pkl','rb')
    self.st_syn_dict=pkl.load(fobj5)
    fobj5.close()
    fobj6=open('cross_index.pkl','rb')
    self.cross_index=pkl.load(fobj6)
    fobj6.close()

  
  def preprocess_query(self,query):# query preprocesser
    query = query.lower()                                        # converting the texts of query string into lowercase
    words = re.sub('[^\w\s\d]',' ',query)                        # removing non-alphanumeric and non-space characters from the query string
    tokens = nltk.word_tokenize(words)                           # tokenizing the modified query string
    words_list = [t for t in tokens if t not in stop_words]      # removing stopwords from the tokens obtained in last step
    for k in self.st_syn_dict.keys():
        for i in range(len(words_list)):
            if words_list[i] == k:
                words_list[i] = self.st_syn_dict[k]
    for k in self.cat_syn_dict.keys():
        for i in range(len(words_list)-1):
            if words_list[i] == k:
                words_list[i] = self.cat_syn_dict[k]
            else:
                key = ' '.join(words_list[i:i+2])
                if key == k:
                    print(self.cat_syn_dict[k])
                    words_list[i] = self.cat_syn_dict[k]
                    words_list[i+1] = ' '
    for k in self.cat_syn_dict.keys():
        if words_list[-1] == k:
            words_list[-1] = self.cat_syn_dict[k]
    words_list = [t for t in words_list if t!=' ']               # removing all blank-space tokens
    if(len(words_list)>10):                                      
      words_list = words_list[:10]
    return words_list

  def make_bigrams(self,words_list):
    index_dict = {word:i for i,word in enumerate(words_list)}# storing position of words in the query
    categories, places, bigram = [],[],[]
    cat_list, city_list, state_list = self.cat_index.get_cat_list(), self.city_index.get_ct_list(), self.state_index.get_state_list()
    for w in words_list:
      if w in cat_list:
        categories.append([w,index_dict[w]])
    for i in range(len(words_list)-1):#finding categories
        cat = ' '.join(words_list[i:i+2])
        if cat in cat_list:
          categories.append([cat,index_dict[words_list[i]]])
    for w in words_list:# finding the words in query which correspond to state or city
      if w in city_list:
        places.append([w,index_dict[w]])
      elif w in state_list:
        places.append([w,index_dict[w]])
    for i in range(len(words_list)-1):# storing phrases which correspond to city/state
        plc = ' '.join(words_list[i:i+2])
        if plc in city_list:
          places.append([plc,index_dict[words_list[i]]])
        elif plc in state_list:
          places.append([plc,index_dict[words_list[i]]])
    categories.sort(key = lambda x: x[1])
    places.sort(key = lambda x: x[1])
    print(categories)
    print(places)
    if len(categories) == len(places):
      for i in range(len(places)):
        self.bigram.append(tuple([categories[i][0],places[i][0]]))# making bigrams on categories places, just based on their locations
    elif len(categories)>0 and len(places)>0:
        for i in range(len(categories)):# making all possible combinations of places and categories
            for j in range(len(places)):
              tup = tuple([categories[i][0],places[j][0]])
              if tup in self.cross_index.cross_index:
                self.bigram.append(tup)
    elif len(categories)>0:# only categories
        for i in range(len(categories)):
            self.bigram.append(categories[i][0])
    elif len(places)>0:
        for i in range(len(places)):# only places
            self.bigram.append(places[i][0])



  def check_place(self,query):
    for i in range(len(df['title'])):
      x = re.findall(df['title'][i], query)
      if len(x) > 0:
        return df.iloc[[i]]
    return None


  def query_processor(self,query):# query processor
    self.res_df = self.check_place(query)
    if self.res_df != None:
      return self.res_df
    self.get_indices()
    words_list = self.preprocess_query(query)
    self.make_bigrams(words_list)
    res_indices = set()
    df = self.cat_index.dataframe
    for bg in self.bigram:# find the list of indices based on the bigrams created
      if type(bg)==tuple():
          res = self.cross_index.index_lookup(bg[0],bg[1])
      else:
          res = self.cross_index.index_lookup(bg)
      res_indices = res_indices.union(res)
    if len(res_indices) > 0 :
      self.res_df = df.iloc[list(res_indices)]
    else:
      self.res_df = df
    self.res_df.drop(['categoryName','address','neighborhood','reviewsTags','totalScore'], axis=1, inplace=True)
    return self.res_df

  

In [ ]:
q = Query_Processing()# creating an object
query = input("Enter a search query : ")
res_df = q.query_processor(query)
res_df

Enter a search query : adventure
['adventure']
[['adventure', 0]]
[]
category adventure
place 


C:\Users\User\AppData\Local\Temp\ipykernel_34232\1776598774.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.res_df.drop(['categoryName','address','neighborhood','reviewsTags','totalScore'], axis=1, inplace=True)


,title,imageUrls,description,city,state
0,Gadeshwar Lake,https://lh5.googleusercontent.com/p/AF1QipNnal...,no information available,maldunge,maharashtra
1,Mandapeshwar Caves,https://lh5.googleusercontent.com/p/AF1QipO0Mr...,"Stone-carved cavern temple, dating to circa 75...",mumbai,maharashtra
3,Chhota Kashmir Boat Club,https://lh5.googleusercontent.com/p/AF1QipOweA...,Laid-back recreational area featuring a lake f...,mumbai,maharashtra
6,Bird Valley Udyan,https://lh5.googleusercontent.com/p/AF1QipNZc7...,"Popular lakeside park offering boating, a jogg...",pimpri-chinchwad,maharashtra
7,Bhayandar Khadi,https://lh5.googleusercontent.com/p/AF1QipOB9E...,no information available,mira bhayandar,maharashtra
...,...,...,...,...,...
3277,"Puliebadze, Jotsoma, Kohima",https://lh5.googleusercontent.com/p/AF1QipMCqD...,no information available,kohima,nagaland
3278,Dzuleke,https://lh5.googleusercontent.com/p/AF1QipMu8G...,no information available,kohima,nagaland
3280,Banyan Trees,https://lh5.googleusercontent.com/p/AF1QipN4sx...,no information available,wokha,nagaland
3283,View point Assam.Nagaland,https://lh5.googleusercontent.com/p/AF1QipM0Ho...,no information available,borjan,nagaland
